# **Visualizing web scraped music festival data in Tableau** 

*By Ryan Moore*

## Project Overview and Objectives

#### *Overview*

The following is a personal project where publicly available tables containing music festival data are web scraped, combined, clean, and visualized in Tableau.

#### *Objectives*

This project will have five main objectives:

1. Web scrape music festival data from Festival Alarm, a publicly available website that tracks music festival data (mainly in Europe)
2. Combine the scraped data into a single table
3. Perform a high-level analysis of the data and clean it as needed
4. Export the data to Tableau and create visualizations for publication
5. Document findings and analysis in a blog post

#### *A note on web scraping*

The Flatiron school data science boot camp teaches it's students to always check the terms and conditions of a website before web scraping. Festival Alarm does not have a terms and conditions page, but it does have a privacy policy page (however, it is in German, so this was translated using Google Translate).

The privacy policy page does not explicitly state that web scraping is not allowed, but it does state that the website is not to be used for commercial purposes.I reached out to the website owners and waited a week for their response; none was given. Since this project is for educational purposes only and there is no explicit statement against web scraping, I decided to proceed with the project.

If anyone from Festival Alarm reads this and would like me to take down this project, please reach out to me and I will do so immediately. You can email me at mooreaz92@gmail.com or message me on GitHub.

## Objective 1: Web scrape the music festival data

**Website Overview**

The target website looks like this:

<div style="text-align: center;">
  <img src="music_festival_viz_practice/images/website_screen_1.png" alt="Website Screen 1" width="50%" style="border-radius: 10px;">
</div>

The above example is for the year of 2023. In order to change years, you must click on the year in the top right corner of the screen. This will bring you to a page that looks like this:

<div style="text-align: center;">
  <img src="music_festival_viz_practice/images/website_screen_2.png" alt="Website Screen 2" width="30%" style="border-radius: 10px;">
</div>

From here, you can click on the year you want to view and a table like the first screenshot will pop up. The table contains the following fields (I have added what I believe each field to mean in italics):

- Name 
  - *The name of the festival*
- Date + [year being viewed] 
  - *The date of the festival*
- Duration 
  - *The duration, in days, of the festival*
- Where 
  - *Whether the festival is indoors or outdoors*
- Category 
  - *The type of music played at the festival*
- Genres 
  - *The genres of music played at the festival*
- Country 
  - *The country the festival is located in*
- Visitors 
  - *The number of visitors the festival had*
- Price 
  - *The price of a ticket to the festival, in euros*
- Links 
  - *Links to the festival's website, ticket site, and an option to add to your festival list*




**Webscraping Strategy**

To webscrape this data, I will use a combination of the `requests` and `BeautifulSoup` libraries. The `requests` library will be used to make the HTTP request to the website and the `BeautifulSoup` library will be used to parse the HTML and extract the data. High level, we will code a custom function that takes in a list of years and that does the following:

1. Makes an HTTP request to the website
2. Parses the HTML using `BeautifulSoup`
3. Extracts the data from the HTML
4. Returns the data in a list of dictionaries
5. Combines the list of dictionaries into a single dataframe

In [1]:
### Importing libraries

import requests
import csv
import pandas as pd
from bs4 import BeautifulSoup

### Doing a sample request to the website, and checking if the request was successful

url = 'https://www.festival-alarm.com/us/Festivals-2023'
response = requests.get(url)
assert response.status_code == 200

**Beautiful Soup Analysis**

Before we can write the webscraping function it would be good if we could get a better understanding of the HTML structure of the website. To do this, we will use the `BeautifulSoup` library to parse the HTML and then use the `prettify()` method to print out the HTML in a more readable format.

In [2]:
### Making a request to the website, and getting the HTML content

url = 'https://www.festival-alarm.com/us/Festivals-2023'
response = requests.get(url)
html = response.content

### Using prettify to make the HTML content more readable, and identifying the html elements that make up the table

soup = BeautifulSoup(html, "lxml")
print(soup.prettify())

<!DOCTYPE html>
<html class="wide wow-animation" lang="en-US">
 <head>
  <style type="text/css">
   .limitdisplay-user { display: none; }.limitdisplay-user-10 { display: inline; }.limitdisplay-user-block-10 { display: block; }
  </style>
  <link href="/extension/festival_alarm/design/festival_alarm/images/favicon/apple-icon-57x57.png" rel="apple-touch-icon" sizes="57x57"/>
  <link href="/extension/festival_alarm/design/festival_alarm/images/favicon/apple-icon-60x60.png" rel="apple-touch-icon" sizes="60x60"/>
  <link href="/extension/festival_alarm/design/festival_alarm/images/favicon/apple-icon-72x72.png" rel="apple-touch-icon" sizes="72x72"/>
  <link href="/extension/festival_alarm/design/festival_alarm/images/favicon/apple-icon-76x76.png" rel="apple-touch-icon" sizes="76x76"/>
  <link href="/extension/festival_alarm/design/festival_alarm/images/favicon/apple-icon-114x114.png" rel="apple-touch-icon" sizes="114x114"/>
  <link href="/extension/festival_alarm/design/festival_alarm/images

After taking a look at the prettified text dump in a seperate text editor, it looks like this table uses the **table**, **tr**, and **td** html table elements. We can use this information when we code the webscraping function.

We can use beautiful soup to pull the table element, and then use the .read_html pandas method to convert the table into a dataframe. This will be useful when we are testing our webscraping function.

In [3]:
### Parsing the HTML content using BeautifulSoup and converting it to a dataframe
table = soup.find('table')

### Converting the HTML table to a dataframe
df = pd.read_html(str(table))[0]

### Printing the info of the dataframe

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 742 entries, 0 to 741
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         742 non-null    object 
 1   Date 2023    742 non-null    object 
 2   Duration     742 non-null    object 
 3   Where        742 non-null    object 
 4   Category     742 non-null    object 
 5   Genres       742 non-null    object 
 6   Country      742 non-null    object 
 7   Venue        742 non-null    object 
 8   Visitors     742 non-null    object 
 9   Price(s.f.)  472 non-null    object 
 10  Unnamed: 10  0 non-null      float64
dtypes: float64(1), object(10)
memory usage: 63.9+ KB


**Checking the sample webscraping method is pulling everything from the site**

Luckily for us, the website has content at the end of the table which shows a count of records. We can use this to check that our webscraping function is pulling all of the data from the website.

<div style="text-align: center;">
  <img src="music_festival_viz_practice/images/website_screen_3.png" alt="Website Screen 3" width="30%" style="border-radius: 10px;">
</div>

Looks like our webscraping function is pulling all of the data from the sample website page. Now we can move on to coding the webscraping function to bring in a bunch of years at once.

## Objective 2: Combine the scraped data into a single table

**Coding the final webscraping function**

Now that we have a webscraping function that works for a single year, we can code a function that will pull in multiple years at once. The function will take in a list of years and will return a dataframe with all of the data from the years in the list. Just for kicks, let's also code a function that will generate a list of urls from a start_year to an end_year.

In [4]:
### Making a function that requests the HTML content of each url in a list of urls and appends each together into a single dataframe

def get_data(urls):
    df_list = []
    for url in urls:
        response = requests.get(url)
        html = response.content
        soup = BeautifulSoup(html, "lxml")
        table = soup.find('table')
        df = pd.read_html(str(table))[0]
        df_list.append(df)
    df = pd.concat(df_list)
    return df

### Making a function that generates Festival Alarm urls for a range of years

def generate_urls(start_year, end_year):
    urls = []
    for year in range(start_year, end_year + 1):
        url = 'https://www.festival-alarm.com/us/Festivals-' + str(year)
        urls.append(url)
    return urls

### Combining the two functions into one

def get_festival_alarm_data(start_year, end_year):
    urls = generate_urls(start_year, end_year)
    df = get_data(urls)
    return df

**Performing the final extract of the data**

Now that we have a webscraping function that works for multiple years, we can use it to extract the data from the website. We will extract all the available data from the website, which is from 2014 to 2024 (future dates for upcoming festivals). We will perform high level checks on the data in the next section.

In [5]:
### Getting the data for the years 2014 to 2024

df = get_festival_alarm_data(2014, 2024)

### Objective 3: Perform a high-level analysis of the data and clean it as needed

With the data extracted, we can now perform a high-level analysis of the data and clean it as needed. 

**Fixing the date column**

From looking at the .info() of the dataframe, we can see that there is an individual date column for each year from 2014 until 2024. Each column contains the start date and end date of the festival, and the column name contains the year of the festival. 

We can fix this by:
- Creating a new column with the year of the festival from the column name
- Creating a 'start_date' and 'end_date' column from the values in the 'Date [year]' column
- Combining the 'start_date' and 'end_date' with the year included

In [6]:
df.columns

Index(['Name', 'Date 2014', 'Duration', 'Where', 'Category', 'Genres',
       'Country', 'Venue', 'Visitors', 'Price(s.f.)', 'Unnamed: 10',
       'Date 2015', 'Date 2016', 'Date 2017', 'Date 2018', 'Date 2019',
       'Date 2020', 'Date 2021', 'Date 2022', 'Date 2023', 'Date 2024'],
      dtype='object')

In [7]:
# Create 'start_date' and 'end_date' columns
df['start_date'] = ''
df['end_date'] = ''

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    start_date = ''
    end_date = ''

    # Iterate over each column in the DataFrame
    for col in df.columns:
        # Check if the column contains 'date' in its name
        if 'date' in col.lower():
            date_range = row[col]  # Get the date range from the current column

            # Check if the date range exists
            if pd.notnull(date_range):
                # Extract start and end dates from the range
                year = col.split()[-1]  # Extract the year from the column name
                start_date = date_range[:5] + ' ' + year
                end_date = date_range[-5:] + ' ' + year
                break  # Exit the loop once a date range is found

    # Update the 'start_date' and 'end_date' columns for the current row
    df.at[index, 'start_date'] = start_date
    df.at[index, 'end_date'] = end_date




**Investigating Nulls**

We should also check for nulls in the dataframe. There is one column that contains only null values; this is likely a column that was added to the website after the data was scraped. We can drop this column.

The remaining data looks good, so we can move on to the next step and create some visualizations to get a high level sense of the data and make sure it makes sense before exporting to Tableau.

**One-hot-encoding the genres column**

The genres column contains a list of genres for each festival. We can one-hot-encode this column to make it easier to work with in Tableau.

**Creating start and end date columns**





In [8]:
df.head()

,Name,Date 2014,Duration,Where,Category,Genres,Country,Venue,Visitors,Price(s.f.),...,Date 2017,Date 2018,Date 2019,Date 2020,Date 2021,Date 2022,Date 2023,Date 2024,start_date,end_date
0,Full Force Festival,07/04 - 07/06,3 days,outdoor,Metal festivals,"Crossover, German rock, Hardcore (Metal), Meta...",Saxony,"Flugplatz Roitzschjora Löbnitz, DE 04509",28000,€ 89.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01/29 2024,02/02 2024
1,Rockharz Festival,07/10 - 07/12,3 days,outdoor,Metal festivals,"German rock, Gothic, Metal, Punk, Rock",Saxony-Anhalt,"Flugplatz Ballenstedt Ballenstedt, DE 06493",12000,€ 85.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03/01 2024,03/02 2024
2,Wacken Open Air,07/31 - 08/02,3 days,outdoor,Metal festivals,"Comedy, German rock, Folk, Gothic, Hardcore (M...",Schleswig-Holstein,"Wacken, DE 25596",75000,€ 170.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03/08 2024,03/09 2024
3,Metal Frenzy Festival,08/14 - 08/16,3 days,outdoor,Metal festivals,"German rock, Gothic, Hardcore (Metal), Metal, ...",Saxony-Anhalt,Erlebnisbad Gardelegen Magdeburger Landstraße ...,1100,€ 59.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03/22 2024,03/24 2024
4,Highfield Festival,08/14 - 08/16,3 days,outdoor,Mixed festivals,"Crossover, German rock, Electro, Hardcore (Met...",Saxony,"Störmthaler See Großpösna, DE 04463",25000,€ 89.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03/28 2024,03/31 2024
